In [2]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel
from dotenv import load_dotenv, find_dotenv
from langchain_together import ChatTogether
from pydantic import BaseModel, Field
import smtplib
from email.message import EmailMessage
import ssl
from twilio.rest import Client
from twilio.twiml.voice_response import VoiceResponse
from decouple import config



In [3]:
load_dotenv(find_dotenv())

True

In [4]:
def make_phone_call(recipient_phone, call_script, twilio_number, account_sid, auth_token):
    """
    Makes a phone call to the recipient and speaks the script using Twilio.
    
    Parameters:
        recipent_phone (str): Recipient's phone number in E.164 format (e.g., "+1234567890").
        call_script (str): The text to be spoken during the call.
        twilio_number (str): Your Twilio phone number (E.164 format).
        account_sid (str): Twilio account SID.
        auth_token (str): Twilio authentication token.
    """
    # Initialize Twilio client
    client = Client(account_sid, auth_token)
    
    # Create TwiML (Telephony Markup Language) to define the call behavior
    twiml_response = VoiceResponse()
    twiml_response.say(call_script, voice='alice')  # 'alice' is a natural-sounding voice
    
    try:
        # Initiate the call
        call = client.calls.create(
            twiml=twiml_response.to_xml(),
            from_=twilio_number,
            to=recipient_phone
        )
        print(f"Call initiated! Call SID: {call.sid}")
    except Exception as e:
        print(f"Failed to make call: {e}")

In [5]:
ACCOUNT_SID = config("TWILIO_ACCOUNT_SID")
AUTH_TOKEN = config("TWILIO_AUTH_TOKEN")
TWILIO_NUMBER = config("TWILIO_PHONE_NUMBER")
RECIPIENT_NUMBER = config("TWILIO_VERIFIED_PHONE_NUMBER")
EMAIL_PASSWORD = config("MAIL_PASSWORD")

In [6]:
# Call the function
make_phone_call(
    recipient_phone=RECIPIENT_NUMBER,
    call_script="Hello! This is an automated call. Thank you for testing our service.",
    twilio_number=TWILIO_NUMBER,
    account_sid=ACCOUNT_SID,
    auth_token=AUTH_TOKEN
)

Failed to make call: HTTP 400 error: Unable to create record: The number +2348067365519 is unverified. Trial accounts may only make calls to verified numbers.


In [6]:
from twilio.twiml.voice_response import VoiceResponse, Say

response = VoiceResponse()
response.say('Hello!')

print(response)

<?xml version="1.0" encoding="UTF-8"?><Response><Say>Hello!</Say></Response>


In [15]:


def send_email(subject, recipient, body, sender_email, sender_password):
    """
    Sends an email using the provided details and SMTP server (e.g., Gmail).
    
    Parameters:
    - subject (str): Email subject
    - recipient (str): Recipient email address
    - body (str): Email body content
    - sender_email (str): Sender's email address (requires SMTP access)
    - sender_password (str): Sender's email password or app-specific password
    """
    # Create the email message
    msg = EmailMessage()
    msg.set_content(body)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = recipient

    # Set up SSL context and SMTP server
    context = ssl.create_default_context()
    
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server:
            server.login(sender_email, sender_password)
            server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")



In [16]:
send_email(
    subject="Hello from Python!",
    recipient="dreemer6@yahoo.com",
    body="This is a test email sent using Python.",
    sender_email="dreemer6.um@gmail.com",
    sender_password=EMAIL_PASSWORD
)

Email sent successfully!


In [17]:
def read_file(file_path):
    """
    Reads a JSON, Excel, CSV, Parquet, Feather, or TSV file into a Pandas DataFrame.
    
    Parameters:
        file_path (str): Path to the file.
    
    Returns:
        pd.DataFrame: DataFrame containing the file data.
    """
    try:
        if file_path.endswith(".csv"):
            return pd.read_csv(file_path)
        elif file_path.endswith(".json"):
            return pd.read_json(file_path)
        elif file_path.endswith(".xlsx") or file_path.endswith(".xls"):
            return pd.read_excel(file_path)
        elif file_path.endswith(".parquet"):
            return pd.read_parquet(file_path)
        elif file_path.endswith(".feather"):
            return pd.read_feather(file_path)
        elif file_path.endswith(".tsv"):
            return pd.read_csv(file_path, sep="\t")
        else:
            raise ValueError("Unsupported file format. Please use JSON, Excel, CSV, Parquet, Feather, or TSV.")
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

In [18]:
def get_company_info(file_path):
    """
    Extracts company-related information from the given file.
    
    Parameters:
        file_path (str): Path to the file.
    
    Returns:
        tuple: Data for Company Names, Fields, Email, Phone, Preferred Channel, Objections, Engagement Level, and Notes.
    """
    df = read_file(file_path)
    if df is None:
        return None
    
    required_columns = [
        "Company_Names", "Company_Fields", "Email_Address", "Phone_Number", 
        "Preferred_Channel", "Objections", "Engagement_Level", "Company_Notes"
    ]
    
    missing_cols = [col for col in required_columns if col not in df.columns]
    
    if missing_cols:
        print(f"Error: Missing columns {missing_cols}.")
        print("Ensure all required columns are present in the file.")
        return None
    
    return (
        df["Company_Names"].values, df["Company_Fields"].values, df["Email_Address"].values, df["Phone_Number"].values, 
        df["Preferred_Channel"].values, df["Objections"].values, df["Engagement_Level"].values, df["Company_Notes"].values
    )

In [19]:
get_company_info('names_and_fields.csv')

(array(['Tech Corp', 'Green Energy', 'MediCare', 'EduWorld'], dtype=object),
 array(['Software', 'Renewable Energy', 'Healthcare', 'Education'],
       dtype=object),
 array(['contact@techcorp.com', 'info@greenenergy.com',
        'support@medicare.com', 'hello@eduworld.com'], dtype=object),
 array(['123-456-7890', '987-654-3210', '555-666-7777', '444-333-2222'],
       dtype=object),
 array(['email', 'phone', 'email', 'phone'], dtype=object),
 array([nan, 'Price concerns', nan, 'Service issues'], dtype=object),
 array(['High', 'Medium', 'Low', 'High'], dtype=object),
 array(['Leading software company', 'Expanding solar solutions',
        'Well-known in medical field', 'Provides online courses'],
       dtype=object))

In [47]:
llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


In [48]:
class EmailAdvise(BaseModel):
    recipient_email: str = Field(description="The **email address of the recipient** (as provided in the input).")
    recipient_phone: str = Field(description="The **phone number of the recipient** (as provided in the input)")
    subject: str = Field(description="A compelling **email subject line** that grabs attention and encourages the recipient to open the email. ")
    email: str = Field(description="A cold email personalized based on the industry, engagement level, and objections.")
    call_script: str = Field(description="A structured cold call script for sales representatives to use.")
    advise: str = Field(description="A strategy to further engage the client based on their industry and previous response.")

In [49]:
parser = JsonOutputParser(pydantic_object=EmailAdvise)

In [53]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"recipient_email": {"description": "The **email address of the recipient** (as provided in the input).", "title": "Recipient Email", "type": "string"}, "recipient_phone": {"description": "The **phone number of the recipient** (as provided in the input)", "title": "Recipient Phone", "type": "string"}, "subject": {"description": "A compelling **email subject line** that grabs attention and encourages the recipient to open the email. ", "title": "Subject", "type": "string"}, "email": {"description": "A cold email personali

In [51]:
prompt = """
You are an AI-powered Cold Outreach Assistant for an insurance company specializing in personalized marketing. Your goal is to craft **highly engaging cold emails** and **concise, high-converting cold call scripts** tailored to prospects' industries, prior engagement levels, and potential objections. Additionally, you will provide strategic follow-up advice to further engage the client.

---

### **Task**  
Given the following details, generate a JSON response containing:  

1. `recipient_email`: The **email address of the recipient** (as provided in the input).  
2. `recipient_phone`: The **phone number of the recipient** (as provided in the input).  
3. `subject`: A compelling **email subject line** that grabs attention and encourages the recipient to open the email.  
4. `email`: A **personalized cold email** that follows this structure:  
   - **[Opening sentence]**: Connect with their industry, a recent trend, or challenge.  
   - **[Value proposition]**: How your insurance solution helps companies in their industry.  
   - **[Objection handling]**: Address a common concern relevant to their industry.  
   - **[CTA]**: Suggest a **quick call, demo, or free consultation**.  

5. `call_script`: A **short but effective cold call script** (under 100 words) that:  
   - Starts with a **brief introduction**.  
   - Highlights a **pain point** relevant to their industry.  
   - Presents the **key benefit of the insurance** in a single sentence.  
   - Handles a **likely objection** in one sentence.  
   - Ends with a **clear CTA** (e.g., booking a call, requesting more info).  

6. `advise`: A **follow-up strategy** to keep the client engaged based on their industry and potential objection.  

---

### **Input**  
- **Recipient Email**: {recipient_email}  
- **Recipient Phone**: {recipient_phone}  
- **Company Name**: {company_name}  
- **Industry**: {industry}  
- **Engagement Level**: {engagement_level}  
- **Potential Objection**: {objection}  
- **Insurance Company Name**: {insurance_company_name}  
- **Sender Name**: {sender_name}  

---

###Note: The email is sent to {company_name} so the greeting at the beginning of the email should be attributed to {company_name}

---

### **Output Format (Strict JSON)**  
{{
  "recipient_email": "{recipient_email}",
  "recipient_phone": "{recipient_phone}",
  "subject": "[Compelling subject line]",
  "email": "[Generated cold email]",
  "call_script": "[Short, high-impact cold call script]",
  "advise": "[Follow-up strategy and recommendations]"
}}


"""  



In [52]:
prompt_template = PromptTemplate(
    template=prompt,
    input_variables=["company_name", "industry", "engagement_level", "objection"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [54]:
first_chain = prompt_template | llm | parser

In [55]:
first_chain

PromptTemplate(input_variables=['company_name', 'engagement_level', 'industry', 'insurance_company_name', 'objection', 'recipient_email', 'recipient_phone', 'sender_name'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"recipient_email": {"description": "The **email address of the recipient** (as provided in the input).", "title": "Recipient Email", "type": "string"}, "recipient_phone": {"description": "The **phone number of the recipient** (as provided in the input)", "title": "Recipient Phone", "type": "st

In [ ]:
response = first_chain.invoke({
    "company_name": "Moniepoint",
    "industry": "Finance",
    "engagement_level": "low",
    "objection": "Security, We’re already insured, Trust with document issues",
    "insurance_company_name": "DigitalInsure",
    "sender_name": "Paul Obiahu",
    "recipient_email": "dreemer6@yahoo.com",
    "recipient_phone": "+2348067365519"
})

In [57]:
response

{'recipient_email': 'paulobiahu2@gmail.com',
 'recipient_phone': '+2348101116037',
 'subject': "Protecting Moniepoint's Financial Future with DigitalInsure",
 'email': "Hi Moniepoint Team,\n\nAs a leading player in the finance industry, you understand the importance of mitigating risks and ensuring business continuity. With the increasing threat of cyber attacks and data breaches, it's crucial to have a robust insurance plan in place.\n\nAt DigitalInsure, we specialize in providing tailored insurance solutions to finance companies like yours. Our policies are designed to protect your business from unforeseen events, ensuring you can focus on growth and innovation.\n\nI understand that security concerns may be top of mind, especially when it comes to document management. Rest assured that our insurance solutions prioritize data protection and confidentiality, giving you peace of mind.\n\nWould you be open to a quick call to discuss how DigitalInsure can help safeguard Moniepoint's finan

In [66]:
def send_cold_email(generated_inputs):
    try:
        send_email(
            subject=generated_inputs['subject'],
            recipient=generated_inputs['recipient_email'],
            body=generated_inputs['email'],
            sender_email="dreemer6.um@gmail.com",
            sender_password=EMAIL_PASSWORD
        )
        return "Success"
    except Exception as e:
        return "Failure"

In [67]:
def send_cold_call(generated_inputs):
    try:
        make_phone_call(
            recipient_phone=generated_inputs['recipient_phone'],
            call_script=generated_inputs['call_script'],
            twilio_number=TWILIO_NUMBER,
            account_sid=ACCOUNT_SID,
            auth_token=AUTH_TOKEN
        )
        return "Success"
    except Exception as e:
        return "Failure"

In [68]:


def get_advise(generated_inputs):
    return generated_inputs['advise']



In [69]:
cold_email_runnable = RunnableLambda(send_cold_email)
cold_call_runnable = RunnableLambda(send_cold_call)
get_advise_runnable = RunnableLambda(get_advise)

In [70]:
email_and_call_chain = RunnableParallel(
    email_status=cold_email_runnable,  
    call_status=cold_call_runnable,  
    advise_status=get_advise_runnable  
)

full_chain = first_chain | email_and_call_chain

In [71]:
response = full_chain.invoke({
    "company_name": "Moniepoint",
    "industry": "Finance",
    "engagement_level": "high",
    "objection": "Security, We’re already insured, Trust with document issues",
    "insurance_company_name": "DigitalInsure",
    "sender_name": "Paul Obiahu",
    "recipient_email": "dreemer6@yahoo.com",
     "recipient_phone": "+2348067365519"
})

Failed to make call: HTTP 400 error: Unable to create record: The number +2348067365519 is unverified. Trial accounts may only make calls to verified numbers.
Email sent successfully!


In [72]:
response

{'email_status': 'Success',
 'call_status': 'Success',
 'advise_status': "Given Moniepoint's high engagement level, I recommend a follow-up email or call within 3-5 days to reiterate the value proposition and address any potential concerns. Since security is a top concern, consider sharing a case study or testimonial from a similar finance company that has benefited from DigitalInsure's insurance solutions. This will help build trust and credibility, increasing the likelihood of conversion."}

In [123]:
prompt = """
You are an AI-powered Cold Outreach Assistant for an insurance company specializing in personalized marketing. Your goal is to craft **highly engaging cold emails** tailored to the input provided. Output only the email content following the provided format. Do not include any extra commentary or explanation.

---

Input: 
- **Prospect's Email**: {prospect_email}  
- **Prospect's name**: {prospect_name}  
- **Prospect's Company Name**: {company_name}  
- **Prospect's Title at company**: {prospect_title}  
- **Industry**: {industry}  
- **Previous Engagement Level**: {engagement_level}  
- **Potential Objection**: {objection}  
- **Outreach Description**: {outreach_description}  
- **Industry Focus**: {industry_focus}  
- **Sender's Company Name**: {insurance_company_name}  
- **Sender's Name**: {sender_name}  
- **Sender's Title at Company**: {sender_title}  

---

###Note: The email is sent to {prospect_name} whose title at {company_name} is {prospect_title} (if provided). So tailor the email accordingly.

### **Output Format (Strict JSON)**
{{
  "prospect_email": "{prospect_email}",
  "subject": "[Compelling subject line]",
  "email": "[Generated cold email]",
  "engagement_advice": "[Follow-up strategy and recommendations]"
}}
"""

In [124]:
# Define Industry-Specific Messaging
from typing import List


def get_industry_focus(industry: str) -> str:
    industry = industry.lower()
    industry_focus_map = {
        "tech": "emphasize innovation and cutting-edge technology solutions.",
        "finance": "highlight security and robust ROI potential.",
        "healthcare": "stress compliance and efficiency.",
    }
    return industry_focus_map.get(industry, "highlight tailored benefits.")


In [125]:
from pydantic import EmailStr


class EmailResponse(BaseModel):
    prospect_email: EmailStr = Field(..., description="The email address of the prospect, as provided in the input.")
    subject: str = Field(..., description="A compelling email subject line generated by the LLM.")
    email: str = Field(..., description="""A **personalized cold email** that follows this structure:  
   - **[Opening sentence]**: Connect with their industry, a recent trend, or challenge.  
   - **[Value proposition]**: Explain how your insurance solution helps companies in their industry.  
   - **[Objection handling]**: Address a common concern relevant to their industry.  
   - **[CTA]**: Suggest a **quick call, demo, or free consultation**. """)
    engagement_advice: str = Field(..., description="A **follow-up strategy** and recommendations to keep the client engaged based on their industry and potential objections.")

# Define JSON Parser

parser = JsonOutputParser(pydantic_object=EmailResponse)

prompt_template = PromptTemplate(
    template=prompt,
    input_variables=[
        "prospect_email",
        "prospect_name",
        "company_name",
        "prospect_title",
        "industry",
        "industry_focus",
        "engagement_level",
        "objections",
        "outreach_description",
        "insurance_company_name",
        "sender_name",
        "sender_title"
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [126]:
prompt_template

PromptTemplate(input_variables=['company_name', 'engagement_level', 'industry', 'industry_focus', 'insurance_company_name', 'objection', 'outreach_description', 'prospect_email', 'prospect_name', 'prospect_title', 'sender_name', 'sender_title'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"prospect_email": {"description": "The email address of the prospect, as provided in the input.", "format": "email", "title": "Prospect Email", "type": "string"}, "subject": {"description": "A compelling email subject lin

In [127]:
from langchain.chat_models import init_chat_model

llama3_70b_llm = init_chat_model( 
    model="llama3-70b-8192", 
    model_provider="groq",
    temperature=0.5,
)

In [128]:
def generate_email_content(params: dict) -> dict:
    # Compute industry focus and add it to parameters.
    params["industry_focus"] = get_industry_focus(params["industry"])
    # Format objections from a list to a comma-separated string.
    # params["objections"] = format_objections(params.get("objections", []))

    print(params)

    # Create a RunnableSequence
    chain = prompt_template | llama3_70b_llm | parser

    response = chain.invoke(params)

    return response

In [129]:
from pprint import pprint
params = {
    "prospect_email": "contact@acme.com",
    "prospect_name": "John Doe",
    "prospect_phone": "+1234567890",
    "company_name": "Acme Corp",
    "prospect_title": "CEO",
    "industry": "tech",
    "engagement_level": 4,
    "objection": "integration complexity",
    "outreach_description": "We offer embedded insurance solutions that reduce risk and drive revenue.",
    "insurance_company_name": "Gamma Insurance Solutions",
    "sender_name": "Alice Johnson",
    "sender_title": "Business Development Manager"
}


# The get_industry_focus function is automatically called within generate_email_content.
output = generate_email_content(params)
pprint(output)


{'prospect_email': 'contact@acme.com', 'prospect_name': 'John Doe', 'prospect_phone': '+1234567890', 'company_name': 'Acme Corp', 'prospect_title': 'CEO', 'industry': 'tech', 'engagement_level': 4, 'objection': 'integration complexity', 'outreach_description': 'We offer embedded insurance solutions that reduce risk and drive revenue.', 'insurance_company_name': 'Gamma Insurance Solutions', 'sender_name': 'Alice Johnson', 'sender_title': 'Business Development Manager', 'industry_focus': 'emphasize innovation and cutting-edge technology solutions.'}
{'email': 'Hi John,\n'
          '\n'
          "As the CEO of Acme Corp, you're no stranger to innovation and "
          "taking bold steps to stay ahead in the tech industry. That's why I "
          'wanted to introduce you to Gamma Insurance Solutions, a company '
          'that shares your passion for cutting-edge technology.\n'
          '\n'
          'Our embedded insurance solutions are designed to reduce risk and '
          'driv

In [130]:
output

{'prospect_email': 'contact@acme.com',
 'subject': 'Innovate with Confidence: Cutting-Edge Insurance Solutions for Acme Corp',
 'email': "Hi John,\n\nAs the CEO of Acme Corp, you're no stranger to innovation and taking bold steps to stay ahead in the tech industry. That's why I wanted to introduce you to Gamma Insurance Solutions, a company that shares your passion for cutting-edge technology.\n\nOur embedded insurance solutions are designed to reduce risk and drive revenue, giving you the confidence to focus on what matters most - pushing the boundaries of what's possible. We've worked with numerous tech companies, helping them navigate complex risks and uncover new opportunities.\n\nI understand that integration complexity can be a concern, but our solutions are built to seamlessly integrate with your existing systems, ensuring a smooth and efficient experience.\n\nI'd love to discuss how Gamma Insurance Solutions can help Acme Corp stay ahead of the curve. Would you be open to a qui

In [ ]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client

# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = os.environ["TWILIO_ACCOUNT_SID"]
auth_token = os.environ["TWILIO_AUTH_TOKEN"]
client = Client(account_sid, auth_token)

call = client.calls.create(
    twiml="<Response><Say>Ahoy, World!</Say></Response>",
    to="+14155551212",
    from_="+15017122661",
)

print(call.sid)

In [8]:
df = pd.read_csv("/home/uchemadu/gamma-cold-chain/processed_outreach_results.csv")
df

,prospect_email,prospect_name,prospect_phone,company_name,prospect_title,industry,engagement_level,objections,outreach_description,insurance_company_name,sender_name,sender_title,outreach_type,subject,email,engagement_advice
0,dreemer6@yahoo.com,John Doe,2348067365519,Acme Corp,CTO,Tech,High,"Pricing, Competitor",Introducing new AI-driven insurance,SecureIns,Jane Smith,Sales Manager,email,Revolutionize Acme Corp's Risk Management with...,"Hi John,\n\nAs the CTO of Acme Corp, you're no...",Follow up with a phone call in 3-4 days to dis...
1,dreemer6@yahoo.com,Alice Brown,2348067365519,Global Fin,CEO,Finance,Medium,Trust Issues,Explaining regulatory compliance benefits,SecureIns,Jane Smith,Sales Manager,call,Protecting Global Fin's Future: Regulatory Com...,"Dear Alice Brown,\n\nAs the CEO of Global Fin,...",Follow up with a phone call in 3-4 days if no ...
2,dreemer6@yahoo.com,Bob Williams,2348067365519,HealthTech Inc,CFO,Healthcare,Low,Budget Constraints,Highlighting cost-saving insurance,SecureIns,Jane Smith,Sales Manager,email,Compliance-Focused Insurance Solutions for Hea...,"Hi Bob,\n\nAs the CFO of HealthTech Inc, you u...","If Bob doesn't respond, consider sending a fol..."
3,dreemer6@yahoo.com,Susan Clark,2348067365519,BuildIt Ltd,Operations Manager,Construction,High,Past Bad Experience,Demonstrating risk coverage policies,SecureIns,Jane Smith,Sales Manager,call,Protect Your Construction Business from Unfore...,"Hi Susan,\n\nAs an Operations Manager at Build...",Follow up with a phone call in 3-4 days if no ...
4,dreemer6@yahoo.com,Michael Lee,2348067365519,AutoDrive,Head of Logistics,Automotive,Medium,Too Expensive,Discussing flexible payment plans,SecureIns,Jane Smith,Sales Manager,email,Streamline Your Logistics with SecureIns,"Hi Michael,\n\nAs the Head of Logistics at Aut...",Follow up with a phone call in 3-4 days if no ...


In [9]:
df1 = pd.read_csv("/home/uchemadu/gamma-cold-chain/uploads/processed_outreach_results.csv")
df1

,prospect_email,prospect_name,prospect_phone,company_name,prospect_title,industry,engagement_level,objections,outreach_description,insurance_company_name,sender_name,sender_title,outreach_type
0,dreemer6@yahoo.com,John Doe,2348067365519,Acme Corp,CTO,Tech,High,"['Pricing', ' Competitor']",Introducing new AI-driven insurance,SecureIns,Jane Smith,Sales Manager,email
1,dreemer6@yahoo.com,Alice Brown,2348067365519,Global Fin,CEO,Finance,Medium,['Trust Issues'],Explaining regulatory compliance benefits,SecureIns,Jane Smith,Sales Manager,call
2,dreemer6@yahoo.com,Bob Williams,2348067365519,HealthTech Inc,CFO,Healthcare,Low,['Budget Constraints'],Highlighting cost-saving insurance,SecureIns,Jane Smith,Sales Manager,email
3,dreemer6@yahoo.com,Susan Clark,2348067365519,BuildIt Ltd,Operations Manager,Construction,High,['Past Bad Experience'],Demonstrating risk coverage policies,SecureIns,Jane Smith,Sales Manager,call
4,dreemer6@yahoo.com,Michael Lee,2348067365519,AutoDrive,Head of Logistics,Automotive,Medium,['Too Expensive'],Discussing flexible payment plans,SecureIns,Jane Smith,Sales Manager,email
